# ETLBox in Polyglot notebooks

## Further links: 
- ETLBox: https://www.etlbox.net
- Polyglot Notebooks: https://devblogs.microsoft.com/dotnet/announcing-polyglot-notebooks-harness-the-power-of-multilanguage-notebooks-in-visual-studio-code
- ScottPlot: https://scottplot.net/quickstart/notebook/ to learn how to setup VS Code and Polyglot extensions
- Microsoft DataFrame: https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/getting-started-dataframe

## Preparation

- Install VS Code
- Install the VS Code Polyglot Notebooks extension

## Running the Example

- Make sure to have an instance of SqlServer available, with a database called "demo".
- Adjust the connection string to your local database in the preparation step.

### Starting the existing notebook

- Open the `.ipynb` with this example, run all code. 

### Create your own fresh new notebook

- Press CTRL + SHIFT + P to open the command dialogue
- Select Polyglot Notebook: Create Default Notebook
- Choose the .ipynb extension and select the C# language
- Add the coe blogs as instructed



In [ ]:

//Step 1: Prepare the database
//Adjust the connection manager to work with your database connection
//A database "demo" must be created before

#r "nuget:ETLBox.SqlServer, 3.4.0"

using ETLBox;
using ETLBox.SqlServer;
using ETLBox.ControlFlow;

Console.WriteLine("Creating demo table with data in database 'demo'!");

IConnectionManager connMan = new SqlConnectionManager("Data Source=localhost;User Id=sa;Password=YourStrong@Passw0rd;Initial Catalog=demo;TrustServerCertificate=true;");
var def = new TableDefinition("Test", new List<TableColumn>() {
    new TableColumn("Id", "INT", allowNulls: false, isIdentity:true, isPrimaryKey:true),
    new TableColumn("XValue", "DATETIME", allowNulls: false),
    new TableColumn("YValue", "INT", allowNulls: false)
});
DropTableTask.DropIfExists(connMan, "Test");
CreateTableTask.CreateIfNotExists(connMan, def);
SqlTask.ExecuteNonQuery(connMan, "INSERT INTO Test VALUES('2022-01-01',100)");
SqlTask.ExecuteNonQuery(connMan, "INSERT INTO Test VALUES('2022-01-02',350)");
SqlTask.ExecuteNonQuery(connMan, "INSERT INTO Test VALUES('2022-01-03',470)");
SqlTask.ExecuteNonQuery(connMan, "INSERT INTO Test VALUES('2022-01-04',134)");
SqlTask.ExecuteNonQuery(connMan, "INSERT INTO Test VALUES('2022-01-05',42)");
Console.WriteLine("Preparation done!");



In [ ]:
//Step 2: Loading data into dataframe and in-memory list

#r "nuget:ETLBox, 3.4.0"
#r "nuget:ETLBox.SqlServer, 3.4.0"
#r "nuget:ETLBox.Analysis, 3.4.0"

using ETLBox;
using ETLBox.ControlFlow;
using ETLBox.SqlServer;
using ETLBox.DataFlow;
using ETLBox.Analysis;

Settings.MaxBufferSize = 10000;

Console.WriteLine("Loading data from database!");
var source = new DbSource(connMan, "Test");
var row = new RowTransformation(row => {
    dynamic r = row as dynamic;
    r.YValue = r.YValue * 1000;
    return row;
});
var multicast = new Multicast();
var memDest = new MemoryDestination();
var dfDest = new DataFrameDestination();
multicast.OnProgress = pc => Console.WriteLine($"Records loaded from database: {pc}");
source.LinkTo(row);
row.LinkTo(multicast);
multicast.LinkTo(memDest);
multicast.LinkTo(dfDest);
Network.Execute(source);

var data = memDest.Data;
var df = dfDest.DataFrame;


In [ ]:
//Displaying the DataFrame
df

In [ ]:
//Transforming the DataFrame
df["x"] = df["YValue"] * 2;
df

In [ ]:
//Using the loaded collection of data to load into a plot
#r "nuget:ScottPlot, 4.1.69"

using Microsoft.DotNet.Interactive.Formatting;

// Setup a custom formatter to display plots as images
Formatter.Register(typeof(ScottPlot.Plot), (p, w) =>
    w.Write(((ScottPlot.Plot)p).GetImageHtml()), HtmlFormatter.MimeType);


var plt = new ScottPlot.Plot(600,400);
var dataX = memDest.Data.Select(row => (DateTime)(row as dynamic).XValue)
                    .Select(dt => dt.ToOADate()).ToArray();
var dataY = memDest.Data.Select(row => (double)(row as dynamic).YValue).ToArray();


plt.XAxis.DateTimeFormat(true);
plt.AddScatter(dataX, dataY);

plt
